<a href="https://colab.research.google.com/github/nerdk312/Model-based-RL/blob/master/Simplified_Model_based_Pacman_ATARI_Adjacent_position_present_290420.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
'''

# Installation and Imports

Used to save Pacman video

In [0]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [17]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!apt-get install ffmpeg

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (46.1.3)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 108 not upgraded.


In [18]:
#!pip install git+git://github.com/mila-iqia/atari-representation-learning.git
!pip install git+git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail
!pip install git+git://github.com/openai/baselines
!pip install wandb

  Cloning git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail to /tmp/pip-req-build-qry3740j
  Running command git clone -q git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail /tmp/pip-req-build-qry3740j
  Created wheel for a2c-ppo-acktr: filename=a2c_ppo_acktr-0.0.1-cp36-none-any.whl size=18833 sha256=8735280c0fb074648041dfd51b2f37348dcd545e7aa81f38280f9b3e4e808499
  Stored in directory: /tmp/pip-ephem-wheel-cache-0xa_3hf1/wheels/91/52/02/ec5c530fd76d56a66934ee91abbdae5240b766be1dc176deb7
Successfully built a2c-ppo-acktr
  Cloning git://github.com/openai/baselines to /tmp/pip-req-build-n2mxoutz
  Running command git clone -q git://github.com/openai/baselines /tmp/pip-req-build-n2mxoutz
  Created wheel for baselines: filename=baselines-0.1.6-cp36-none-any.whl size=220664 sha256=13c192478491c9c527ed6b5619db4281bd5d575668dc0709c9404fd201a5f544
  Stored in directory: /tmp/pip-ephem-wheel-cache-90rqojqr/wheels/42/1c/91/28314e0cd1d2cc57cf8dd18b20c4c9a0f39ae518adc13caf24
Successfully

In [19]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
from __future__ import print_function
import pickle
import sys
sys.path.append('/content/gdrive/My Drive/Unsupervised_state_representation/atariari')

import wandb

import random
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np

from tqdm import tqdm
import datetime
import time

from benchmark.envs import *
from torch.autograd import Variable

import gym
from benchmark.wrapper import AtariARIWrapper

#from benchmark import *
#from methods import utils

# Needed to create dataloaders
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from custom.EarlyStopping import EarlyStopping_loss
from custom.model import *
from custom.custom_wrappers import DeterminsticNoopResetEnv 
# Imported required for the Model-based RL
#from sklearn.preprocessing import StandardScaler
from baselines.common.atari_wrappers import EpisodicLifeEnv

In [21]:
!wandb login ################

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


# General Functions

In [0]:
class General_functions():
    def __init__(self, ENV_NAME,n_actions,possible_positions):
        self.ENV_NAME = ENV_NAME
        self.env = gym.make(self.ENV_NAME)
        self.env = EpisodicLifeEnv(DeterminsticNoopResetEnv(self.env,66))        
        self.env = AtariARIWrapper(self.env)
        self.initial_info_labels = self.env.labels()
    
        self.prev_action_counter =  False
        self.repeated_end = False # Nawid -  Sets self.repeated end as false initially
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.n_actions = n_actions
        # possible positions is numpy array with most of the possible positions that the agent can go to
        self.possible_positions = possible_positions
        self.possible_positions_list = self.possible_positions.tolist()

    def one_hot(self,i):
        a = np.zeros(self.n_actions, 'uint8')
        a[i-1] = 1
        return a

    def state_conversion(self,obs,info_labels):
        state = []          
        for key in info_labels:
            if key == 'player_x' or key == 'player_y': # Nawid - Only the first 14 info are crucial I believe
                    state.append(info_labels[key])
                    
        state = np.array(state)
        return state
    
    def next_position(self,state, action):
        next_position = state.copy()
        if action == 1:
            next_position[1] = next_position[1] - 2 
        elif action == 2:
            next_position[0] = next_position[0] + 2
        elif action == 3:
            next_position[0] = next_position[0] - 2
        elif action == 4:
            next_position[1] = next_position[1] + 2
        
        if next_position.tolist() in self.possible_positions_list: # possible positions will be a list which is fed into the network 
            return True
        else:
            return False
            
    ''' This is a general case which can be used with both the controller and the data collector
    def random_action_selection(self, state,prev_action = None):
        while True:
            action = np.random.randint(1,5) 
            feasible_action = self.next_position(state,action)
            if feasible_action:
                self.prev_action_counter = True
                #print('action:',action)
                return action
            else:
                if self.prev_action_counter and prev_action !=None:
                    #print('repeated action:',prev_action)
                    return prev_action
    '''

    def available_positions(self,current_position):
        availability = np.zeros(self.n_actions, 'uint8')
        

        action1_scenario = current_position.copy()
        
        #print('action 1 scenario:',action1_scenario)
        #print('action 1 scenario shape:',action1_scenario.shape)
        action1_scenario[1] = action1_scenario[1] - 2

        action2_scenario = current_position.copy()
        action2_scenario[0] = action2_scenario[0] + 2

        action3_scenario = current_position.copy()
        action3_scenario[0] = action3_scenario[0] - 2

        action4_scenario = current_position.copy()
        action4_scenario[1] = action4_scenario[1] + 2

        if action1_scenario.tolist() in self.possible_positions_list:
            availability[0] = True
        else:
            availability[0] = False
        
        if action2_scenario.tolist() in self.possible_positions_list:
            availability[1] = True
        else:
            availability[1] = False
         
        if action3_scenario.tolist() in self.possible_positions_list:
            availability[2] = True
        else:
            availability[2] = False
        
        if action4_scenario.tolist() in self.possible_positions_list:
            availability[3] = True
        else:
            availability[3] = False
        
        return availability

    
    def check_all_agents(self,info_label,next_info_label):
        repeated = np.equal(info_label,next_info_label).all()
        if repeated:
            self.repeated_end= True

    def check_state(self,state, next_state):
        repeated_state = np.equal(state, next_state).all()
        if repeated_state:
            self.prev_action_counter = False


# Data collection and preprocessing

In [23]:
class Data_collection(General_functions):
    def __init__(self,ENV_NAME,n_actions, possible_positions):
        super(Data_collection,self).__init__(ENV_NAME,n_actions,possible_positions)
        self.dataset_random = [] # Where the data is saved each time

    def collate_data(self,random_dataset, rl_dataset):
        rand_data = np.array(random_dataset)
        num_rand_examples = len(rand_data)
        D_train = rand_data[:int(-num_rand_examples*1/5)] 
        D_valid = rand_data[int(-num_rand_examples*1/5):]
        print("number random examples:",num_rand_examples, 'len(D_train_rand)', len(D_train),'len(D_valid_rand)', len(D_valid))
        if len(rl_dataset) > 0:
            # Adds the rl dataset to the random dataset if there is any present
            rl_data = np.array(rl_dataset)
            num_rl_examples = len(rl_data)
            D_rl_train = rl_data[:int(-num_rl_examples*1/5)] 
            D_rl_valid = rl_data[int(-num_rl_examples*1/5):]
                        
            D_train = np.concatenate([D_train, D_rl_train], axis = 0)
            D_valid = np.concatenate([D_valid, D_rl_valid], axis = 0)
            print("number rl examples:",num_rl_examples, 'len(D_rl_train)', len(D_rl_train),'len(D_valid_rand)', len(D_rl_valid))
            
        #print("len(D_train):", len(D_train), 'len(D_valid)', len(D_valid))

        # Shuffle the dataset
        '''  
        sff = np.arange(len(D_train))
        np.random.shuffle(sff)
        D_train = D_train[sff]
        '''
        #print('D_train shape',D_train.shape)

        # Create the input and output for the train
        X_train_obs = np.array([obs for obs,_,_,_,_,_ in D_train]) # Takes obs and action
        X_train_obs = X_train_obs.astype(np.int16) # Need to change it to a int16 so it is signed ( so negative values can be calculated)
        X_train_avail = np.array([avail for _,_,_,_,_,avail in D_train]) 
        X_train_act = np.array([act for _,_,_,_,act,_ in D_train])        

        # Env output
        y_env_train = np.array([no for _,no,_,_,_,_ in D_train])
        y_env_train = y_env_train.astype(np.int16) # Need to change it to a int16 so it is signed ( so negative values can be calculated)
        y_env_train = y_env_train - X_train_obs 
        
        X_train_obs = np.concatenate((X_train_obs, X_train_avail),axis=1) # Full state after appending the domain knowledge
        
        # Next state output
        X_val_obs = np.array([obs for obs,_,_,_,_,_ in D_valid]) # Takes obs and action
        X_val_obs = X_val_obs.astype(np.int16) # Need to change it to a int16 so it is signed ( so negative values can be calculated)        
        X_val_avail =  np.array([avail for _,_,_,_,_,avail in D_valid])
        X_val_act = np.array([act for _,_,_,_,act,_ in D_valid])

        y_env_val = np.array([no for _,no,_,_,_,_ in D_valid])
        y_env_val = y_env_val.astype(np.int16)
        y_env_val = y_env_val - X_val_obs 

        X_val_obs = np.concatenate((X_val_obs, X_val_avail),axis=1)

        env_train_data, env_val_data = (X_train_obs, X_train_act, y_env_train), (X_val_obs, X_val_act, y_env_val)
        return env_train_data, env_val_data 

    def setup_dataset(self,env_train_data, env_val_data):
        # Unpack data
        (X_env_train_obs, X_env_train_act, y_env_train), (X_env_val_obs, X_env_val_act, y_env_val) = env_train_data, env_val_data
    
        # Concatentates the normalised states with the one hot vector for the actions
        X_env_train = np.concatenate((X_env_train_obs,X_env_train_act),axis=1)
        X_env_val = np.concatenate((X_env_val_obs,X_env_val_act),axis=1)

        # Pack data tuples
        env_train_data, env_val_data = (X_env_train, y_env_train),(X_env_val, y_env_val) 
        return env_train_data, env_val_data
    
    def random_action_selection(self, state,prev_action = None):
        while True:
            action = np.random.randint(1,5) 
            feasible_action = self.next_position(state,action)
            if feasible_action:
                self.prev_action_counter = True
                #print('action:',action)
                return action
            else:
                infeasible_action_one_hot = self.one_hot(action)
                availability_dummy = self.available_positions(state)
                self.dataset_random.append([state, state, 0, False, infeasible_action_one_hot, availability_dummy]) # NEED TO CHANGE WHERE THE DATA IS SAVED WHEN I AM USING THE DONES SINCE THE DONES ARE ADDED IN THE MAIN FUNCTION TO PAST VALUES   
                if self.prev_action_counter and prev_action !=None:
                    #print('repeated action:',prev_action)
                    return prev_action
    

    def gather_random_trajectories(self,num_traj):
        #dataset_random = []
        for n in range(num_traj):
            if n % 10 ==0:
                print('trajectory number:',n)
                # Initial set up
            #self.env.seed(0)
            self.env = gym.make(self.ENV_NAME)
            self.env = EpisodicLifeEnv(DeterminsticNoopResetEnv(self.env,noop_max=66))
            self.env = AtariARIWrapper(self.env)
            obs = self.env.reset()
            
            self.repeated_end = False
            info_labels = self.env.labels() # Nawid - Used to get the current state
            state = self.state_conversion(obs, info_labels) # Used to get the initial state
            prev_action = None # Initialise prev action has having no action

            while True:
                sampled_action = self.random_action_selection(state,prev_action)
                #sampled_action = np.random.randint(1,5)
                availability = self.available_positions(state) 
                #print('availability',availability.shape)

                sampled_action_one_hot = self.one_hot(sampled_action)
                next_obs, reward, done, next_info = self.env.step(sampled_action)
                next_info_labels = next_info['labels']
                
                next_state = self.state_conversion(next_obs, next_info_labels)
                self.check_state(state,next_state)
                self.check_all_agents(info_labels, next_info_labels) # need to use the info labels to predict the state as the info labels have all the informaiton
                    
                if not self.repeated_end:
                    self.dataset_random.append([state, next_state, reward, done,sampled_action_one_hot,availability])

                else:
                    self.dataset_random[-1][-3] = True # sets the previous value to done
                    done = True   
                    
                state = next_state.copy()
                info_labels = next_info_labels.copy()
                prev_action = sampled_action
                if done:
                    break    
        
        return self.dataset_random    
'''
ENV_NAME = 'MsPacman-ramDeterministic-v4'
n_actions = 4 #9 - Nawid - Change to 5 actions as the 4 other actions are simply copies of the other actions, therefore 5 actions should lower the amount of data needed.

possible_positions = np.load('/content/gdrive/My Drive/MsPacman-data/possible_pacman_positions.npy',allow_pickle=True)
data_object = Data_collection(ENV_NAME,n_actions,possible_positions)
data1 = data_object.gather_random_trajectories(10)

rl_dataset = []
env_train, env_val = data_object.collate_data(data1,rl_dataset)
full_env_train, full_env_val = data_object.setup_dataset(env_train,env_val)
x_train,y_train = full_env_train
x_train.shape
'''

"\nENV_NAME = 'MsPacman-ramDeterministic-v4'\nn_actions = 4 #9 - Nawid - Change to 5 actions as the 4 other actions are simply copies of the other actions, therefore 5 actions should lower the amount of data needed.\n\npossible_positions = np.load('/content/gdrive/My Drive/MsPacman-data/possible_pacman_positions.npy',allow_pickle=True)\ndata_object = Data_collection(ENV_NAME,n_actions,possible_positions)\ndata1 = data_object.gather_random_trajectories(10)\n\nrl_dataset = []\nenv_train, env_val = data_object.collate_data(data1,rl_dataset)\nfull_env_train, full_env_val = data_object.setup_dataset(env_train,env_val)\nx_train,y_train = full_env_train\nx_train.shape\n"

# Controller

In [0]:
class multi_model_based_control(General_functions):
    def __init__(self,ENV_NAME,n_actions,possible_positions,env_model,num_sequences,horizon_length):
        super(multi_model_based_control,self).__init__(ENV_NAME,n_actions,possible_positions)
        self.env = gym.wrappers.Monitor(self.env, wandb.run.dir, video_callable=lambda episode_id: True if episode_id % 2 ==0 else False, force= True)
        self.env_model = env_model
        self.horizon_length = horizon_length
        self.num_sequences = num_sequences        
        self.n_actions = n_actions
        self.global_step = 0
        self.rl_dataset = []

    def random_sampling_shooting(self,real_obs):
        '''
        Use a random-sampling shooting method, generating random action sequences. The first action with the highest reward of the entire sequence is returned
        '''
        best_reward = -1e9
        best_next_action = []
        m_obs = np.array([real_obs for _ in range(self.num_sequences)])
        
        # array that contains the rewards for all the sequence
        unroll_rewards = np.zeros((self.num_sequences, 1)) 
        first_sampled_actions = []
        
        
        # Create a batch of size 'num_sequences' (number of trajectories) to roll the models 'horizon length' times
        ## i.e roll a given number of trajectories in a single batch (to increase speed)
        for t in range(self.horizon_length):
            # sample actions for each sequence
            availability = np.array([self.available_positions(real_obs)])
            #print('availability:',availability)
            #print('availability shape:', availability.shape)
            #availability = self.available_positions(real_obs) # try m_obs next
        

            sampled_actions = np.array([np.random.randint(1,5) for _ in range(self.num_sequences)])
            sampled_actions_one_hot = np.array([self.one_hot(action) for action in sampled_actions])
            #print('sampled_actions_one_hot:', sampled_actions_one_hot)
            env_model_input = np.concatenate([m_obs,availability,sampled_actions_one_hot], axis = 1)
            #print('env_model input:',env_model_input.shape)            
            # compute the next state for each sequence
            pred_obs = self.env_model(torch.tensor(env_model_input).to(self.device))
            print('m obs:',m_obs)
            print('sampled_action:',sampled_actions)
            print('availability:', availability)
            print('pred_obs:', pred_obs)


            # add previous observation
            next_obs = pred_obs.cpu().detach().numpy() + m_obs
            #print('next observation',next_obs)

            #print('player',m_obs[:,8:10])
        
            m_obs = next_obs # Nawid - Update the state after calculating the new state and calculating the rewards

            if t ==0:
                first_sampled_actions = sampled_actions 


        # Best the position of the sequence with the higher reward
        
        random_choice = np.random.randint(0,self.num_sequences)

        # take the first action of this sequence
        best_action = first_sampled_actions[random_choice]
        #print('best action',best_action)
        #best_action =  np.squeeze(best_action)
        return best_action
    
    def on_policy_collection(self,STEPS_PER_AGGR):
        obs = self.env.reset()
        self.env.seed(0) # Set the random seed of the environment
        num_examples_added = 0

        # records how long the agent survives for
        timesteps = 0
        wait_done = False # Use to wait until the episode ends after the agent has lot all of its lives and has been eaten
        # records how many simulations have occurred
        
        info_labels = self.env.labels()
        state = self.state_conversion(obs,info_labels)
        next_state = state.copy() # Initially the state and the next state will be the same
        while num_examples_added < STEPS_PER_AGGR:
            while True:
                                                
                state = self.state_conversion(obs,info_labels)
                action = self.random_sampling_shooting(state)
                    
                action_one_hot = self.one_hot(action)
                # one step in the environment with the action returned by
                next_obs, reward, done, next_info = self.env.step(action)
                next_info_labels = next_info['labels']                
                next_state = self.state_conversion(next_obs, next_info_labels)
                self.rl_dataset.append([state, next_state, reward, done, action_one_hot])
                                
                obs = next_obs
                info_labels = next_info_labels
                num_examples_added += 1 

                if done:
                    obs = self.env.reset()
                    info_labels = self.env.labels()
                    self.env.seed(0)
                    break
        return self.rl_dataset  

# Hyperparameters

In [25]:
wandb.init(entity="nerdk312", project="ATARI_LABELS_MPC")#,monitor_gym=True)
ENV_NAME = 'MsPacman-ramDeterministic-v4' #'MsPacmanNoFrameskip-v4'  #'MsPacmanDeterministic-v4' # 'MsPacmanNoFrameskip-v4'
 
# Main loop hyperp
env_model_pretrain = False
pretrained_env_model = '/content/gdrive/My Drive/MsPacman-data/Env model/Env_model_21_4_2020-8_37.pt'#'/content/gdrive/My Drive/MsPacman-data/Env model/Env_model_17_4_2020-11_0.pt' #'/content/Env_model_17_4_2020-10_32.pt' #'/content/Epoch_no_43_Env_model_13_4_2020-8_22_checkpoint.pt'
ENV_LEARNING_RATE = 1e-4

AGGR_ITER = 10
STEPS_PER_AGGR = 1000

# Random MB hyperp
NUM_RAND_TRAJECTORIES = 6000

# 'cuda' or 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
# Supervised Model Hyperp
BATCH_SIZE = 1024
TRAIN_ITER_MODEL =  10000

# Controller Hyperp
HORIZON_LENGTH = 1
NUM_ACTIONS_SEQUENCES = 1

load_data = True
if load_data:
    loaded_trajectories = '/content/gdrive/My Drive/MsPacman-data/rand_traj_6000-29_4_2020-15_30_added_adjacent_pos.npy' #'/content/gdrive/My Drive/MsPacman-data/only_agent_position_6000_traj__prev_action_fix+failed_action_fix.npy' #'/content/gdrive/My Drive/MsPacman-data/rand_traj_3000-29_4_2020-9_8.npy' #'/content/gdrive/My Drive/MsPacman-data/only_agent_position_6000_traj_allowed_move_prev_action_fix.npy' #'/content/gdrive/My Drive/MsPacman-data/rand_traj_3000-28_4_2020-18_0.npy' #'/content/gdrive/My Drive/MsPacman-data/only_agent_position_3000_traj_rand+allowed_moves.npy' #'/content/gdrive/My Drive/MsPacman-data/rand_traj_3000-21_4_2020-7_47.npy'#'/content/gdrive/My Drive/MsPacman-data/Random trajectories/No_wall_movements/rand_traj_3000-_all_lives_16_4_2020-14_33.npy'#'/content/gdrive/My Drive/MsPacman-data/rand_traj_3000-16_4_2020-14_33.npy'#

collect_data = False

observation_channels = 1
action_dim = 1
n_actions = 4 #9 - Nawid - Change to 5 actions as the 4 other actions are simply copies of the other actions, therefore 5 actions should lower the amount of data needed.
reward_dim = 1

# Time and date information
now = datetime.datetime.now()
date_time = "{}_{}_{}-{}_{}".format(now.day,now.month,now.year, now.hour, now.minute)

possible_positions = np.load('/content/gdrive/My Drive/MsPacman-data/possible_pacman_positions.npy',allow_pickle=True)
# Making the network deterministic - https://pytorch.org/docs/stable/notes/randomness.html
random_seed = 0

if random_seed:    
    torch.manual_seed(1)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    #np.random.seed(0)

cuda


In [0]:
config = wandb.config
config.batch_size = BATCH_SIZE          
config.horizon_length = HORIZON_LENGTH
config.num_action_seq = NUM_ACTIONS_SEQUENCES
config.num_rand_trajectories = NUM_RAND_TRAJECTORIES
config.aggr_iter = AGGR_ITER
config.steps_per_aggr = STEPS_PER_AGGR

config.train_model_iter = TRAIN_ITER_MODEL
config.env_lr = ENV_LEARNING_RATE
config.env_model_pretrain = env_model_pretrain
config.pretrained_env =  pretrained_env_model

config.no_actions = n_actions
config.load_data = load_data
config.collect_data = collect_data
config.random_seed = random_seed

if load_data:
    config.loaded_trajectories = loaded_trajectories

# Main - Model training

Data collection and processing

In [27]:
# gather the dataset of random sequences
data_collector = Data_collection(ENV_NAME,n_actions,possible_positions)
if load_data:
    rand_dataset = np.load(loaded_trajectories,allow_pickle=True)
else:
    rand_dataset = data_collector.gather_random_trajectories(NUM_RAND_TRAJECTORIES)
    filename = '/content/gdrive/My Drive/MsPacman-data/rand_traj_{}-'.format(NUM_RAND_TRAJECTORIES) + date_time
    np.save(filename,rand_dataset)

#rand_dataset = np.array([x for x in rand_dataset if not same(x)])
#rand_dataset = rand_dataset[0:100,:]
empty_dataset = []
env_train_data, env_val_data = data_collector.collate_data(rand_dataset,empty_dataset)

norm_env_train_data, norm_env_val_data = data_collector.setup_dataset(env_train_data, env_val_data)

number random examples: 885729 len(D_train_rand) 708584 len(D_valid_rand) 177145


Dynamics model training

In [28]:
env_model = NNDynamicsModel(n_actions + 6, 2).to(device) # Nawid - Need to put both actions as it is a one-hot vector

if env_model_pretrain:
    print('checkpoint')
    #env_model.load_state_dict(torch.load(pretrained_env_model))
    checkpoint = torch.load(pretrained_env_model)
    env_model.load_state_dict(checkpoint['model_state_dict'])
    
    for parameter in env_model.parameters():
        parameter.requires_grad = False
    env_model.eval()

else:
    env_optimizer = torch.optim.Adam(env_model.parameters(),ENV_LEARNING_RATE) # Nawid - Optimizer for the env model
    wandb.watch(env_model, log="all")
    env_model_name = 'Env_model'+ '_' + date_time
    early_stopping_env = EarlyStopping_loss(patience=5, verbose=True, wandb=wandb, name=env_model_name)
    for n_iter in range(AGGR_ITER):
        if early_stopping_env.early_stop:
            print('Early stopping')
            break 
        train_model(norm_env_train_data, norm_env_val_data, env_model, BATCH_SIZE, TRAIN_ITER_MODEL,env_optimizer,device,early_stopping_env)

#rew_model = Reward_Oracle()

  0%|          | 1/10000 [00:10<29:20:21, 10.56s/it]

Validation loss decreased/improved for Env_model_29_4_2020-15_47  (100000000000.000000 --> 0.145661).  Saving model ...
epoch: 0 completed


  0%|          | 2/10000 [00:22<30:21:23, 10.93s/it]

Validation loss decreased/improved for Env_model_29_4_2020-15_47  (0.145661 --> 0.073809).  Saving model ...
epoch: 1 completed


  0%|          | 3/10000 [00:33<30:09:12, 10.86s/it]

...

EarlyStopping for Env_model_29_4_2020-15_47 counter: 2 out of 5
epoch: 9 completed


  0%|          | 11/10000 [01:58<29:36:46, 10.67s/it]

EarlyStopping for Env_model_29_4_2020-15_47 counter: 3 out of 5
epoch: 10 completed


  0%|          | 12/10000 [02:09<29:36:27, 10.67s/it]

EarlyStopping for Env_model_29_4_2020-15_47 counter: 4 out of 5
epoch: 11 completed
EarlyStopping for Env_model_29_4_2020-15_47 counter: 5 out of 5
Env_model_29_4_2020-15_47 has stopped
Early stopping
Early stopping


Training the reward function and MPC

In [29]:

#rand_env_model = NNDynamicsModel(n_actions + 6, 2).to(device) # Nawid - Need to put both actions as it is a one-hot vector
#rand_env_model.eval()
env_model.eval()
Controller = multi_model_based_control(ENV_NAME,n_actions,possible_positions, env_model, NUM_ACTIONS_SEQUENCES, HORIZON_LENGTH)
for n_iter in range(AGGR_ITER):    
    '''
    if early_stopping_env.early_stop:~
        print('Early stopping')
        break 
    '''
    #train_model(norm_env_train_data, norm_env_val_data, env_model, BATCH_SIZE, TRAIN_ITER_MODEL,env_optimizer,device,early_stopping_env)    
    rl_dataset = Controller.on_policy_collection(100000)

Controller.env.close()
    
'''
    if collect_data:
        env_train_data, env_val_data, rew_train_data, rew_val_data = data_collector.collate_data(rand_dataset,rl_dataset)
        norm_env_train_data, norm_env_val_data, norm_rew_train_data, norm_rew_val_data = data_collector.setup_dataset(env_train_data, env_val_data, rew_train_data, rew_val_data)
'''

Streaming output truncated to the last 5000 lines.
sampled_action: [1]
availability: [[0 1 1 0]]
pred_obs: tensor([[0.0070, 0.0061]], device='cuda:0', grad_fn=<AddmmBackward>)
m obs: [[101  98]]
sampled_action: [3]
availability: [[0 1 1 0]]
pred_obs: tensor([[-2.8184, -0.0073]], device='cuda:0', grad_fn=<AddmmBackward>)
m obs: [[99 98]]
sampled_action: [3]
availability: [[0 1 1 0]]
pred_obs: tensor([[-2.8091, -0.0072]], device='cuda:0', grad_fn=<AddmmBackward>)
m obs: [[97 98]]
sampled_action: [2]
availability: [[0 1 1 0]]
pred_obs: tensor([[2.7353e+00, 5.4799e-04]], device='cuda:0', grad_fn=<AddmmBackward>)
m obs: [[100  98]]
sampled_action: [3]
availability: [[0 1 1 0]]
pred_obs: tensor([[-2.8138, -0.0070]], device='cuda:0', grad_fn=<AddmmBackward>)
m obs: [[97 98]]
sampled_action: [2]
availability: [[0 1 1 0]]
pred_obs: tensor([[2.7353e+00, 5.4799e-04]], device='cuda:0', grad_fn=<AddmmBackward>)
m obs: [[100  98]]
sampled_action: [2]
availability: [[0 1 1 0]]
pred_obs: tensor([[2.73

KeyboardInterrupt: ignored

# Old Code

In [0]:
loaded_trajectories = '/content/gdrive/My Drive/MsPacman-data/rand_traj_3000-29_4_2020-9_8.npy'
loaded_dataset = np.load(loaded_trajectories,allow_pickle=True)

loaded_trajectory_2 = '/content/gdrive/My Drive/MsPacman-data/rand_traj_3000-29_4_2020-9_50.npy'
loaded_dataset2 = np.load(loaded_trajectory_2,allow_pickle=True)

collated_dataset = np.concatenate((loaded_dataset,loaded_dataset2),axis=0)
filename = '/content/gdrive/My Drive/MsPacman-data/only_agent_position_6000_traj__prev_action_fix+failed_action_fix'
np.save(filename,collated_dataset)